<a id='101-bullet'></a>
# Анализ данных сервиса Яндекс.Дзен

Подготовим данные для визуализации данных сервиса. Проанализируем взаимодействие пользователей с карточками статей.
Каждую карточку определяют её тема и источник (у него тоже есть тема). Примеры тем: «Красота и здоровье», «Россия», «Путешествия». Пользователей системы характеризует возрастная категория. Скажем, «26-30» или «45+».
_______________________________________________________________________________________________________________
**Есть три способа взаимодействия пользователей с системой:**
1. Карточка отображена для пользователя (show);
2. Пользователь кликнул на карточку (click);
3. Пользователь просмотрел статью карточки (view).
_______________________________________________________________________________________________________________
**Дашборд должен ответить на следующие вопросы:**
1. Сколько взаимодействий пользователей с карточками происходит в системе с разбивкой по темам карточек?
2. Как много карточек генерируют источники с разными темами?
3. Как соотносятся темы карточек и темы источников?
_______________________________________________________________________________________________________________
**Техническое задание на реализацию дашборда.**
1. Бизнес-задача: анализ взаимодействия пользователей с карточками Яндекс.Дзен;
2. Насколько часто предполагается пользоваться дашбордом: не реже, чем раз в неделю;
3. Кто будет основным пользователем дашборда: менеджеры по анализу контента;
4. Состав данных для дашборда:
   - История событий по темам карточек (два графика - абсолютные числа и процентное соотношение);
   - Разбивка событий по темам источников;
   - Таблица соответствия тем источников темам карточек;
5. По каким параметрам данные должны группироваться:
   - Дата и время;
   - Тема карточки;
   - Тема источника;
   - Возрастная группа;
6. Характер данных:
   - История событий по темам карточек — абсолютные величины с разбивкой по минутам;
   - Разбивка событий по темам источников — относительные величины (% событий);
   - Соответствия тем источников темам карточек - абсолютные величины;
7. Важность: все графики имеют равную важность;
8. Источники данных для дашборда: cырые данные о событиях взаимодействия пользователей с карточками (таблица log_raw);
9. База данных, в которой будут храниться агрегированные данные: дополнительные агрегированные таблицы в БД zen;
10. Частота обновления данных: один раз в сутки, в полночь по UTC;
11. Какие графики должны отображаться и в каком порядке, какие элементы управления должны быть на дашборде (макет дашборда):
![title](dash_tt.jpg)
___________________________   
***Описание данных:***

Датафрейм dash_visits:
1. record_id - идентификатор карточки;
2. item_topic - тема карточки;
3. source_topic - тема источника;
4. age_segment - возрастная категория;
5. dt - дата и время события;
6. visits - количество визитов;

Датафрейм log_raw:
1. event_id - идентификатор события;
2. age_segment - возрастная категория;
3. event - наименование события;
4. item_id - идентификатор карточки;
5. item_topic - тема карточки;
6. item_type - тип карточки;
7. source_id - идентификатор источника; 
8. source_topic - тема источника;
9. source_type - тип источника;
10. ts - дата и время;
11. user_id - идентификатор пользователя.

## Подключение к базе, извлечение и сохранение данных

In [4]:
#загрузим необходимые библиотеки
import pandas as pd
from sqlalchemy import create_engine

#создадим подключение к базе
db_config = {'user': 'praktikum_student', # имя пользователя
            'pwd': 'Sdf4$2;d-d30pp', # пароль
            'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
            'port': 6432, # порт подключения
            'db': 'data-analyst-zen-project-db'} # название базы данных

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                db_config['pwd'],
                                                db_config['host'],
                                                db_config['port'],
                                                db_config['db'])

try:
    engine = create_engine(connection_string)
    print('Подключение успешно создано')
except:
    print('Не удалось создать подключение')
    
    
#выгрузим данные dash_visits и сохраним их в файле csv
try:
    query = '''
    select * from dash_visits
    '''
    dash_visits = pd.io.sql.read_sql(query, con = engine)
    dash_visits.to_csv('dash_visits.csv')
    print('Файл "dash_visits.csv" успешно сохранен')
except:
    print('Не удалось сохранить файл')

Подключение успешно создано
Файл "dash_visits.csv" успешно сохранен


## Проверим данные

In [3]:
#проверим данные на наличие дубликатов и пропусков
print('В датафрейме dash_visits {} дубликатов'.format(dash_visits.duplicated().sum()))
print()
print('Датафрейм содержит информацию за период с {} по {}'.format(dash_visits.dt.min(), dash_visits.dt.max()))
print()
print(dash_visits.info())
display(dash_visits.sample(7))

В датафрейме dash_visits 0 дубликатов

Датафрейм содержит информацию за период с 2019-09-24 18:28:00 по 2019-09-24 19:00:00

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30745 entries, 0 to 30744
Data columns (total 6 columns):
record_id       30745 non-null int64
item_topic      30745 non-null object
source_topic    30745 non-null object
age_segment     30745 non-null object
dt              30745 non-null datetime64[ns]
visits          30745 non-null int64
dtypes: datetime64[ns](1), int64(2), object(3)
memory usage: 1.1+ MB
None


,record_id,item_topic,source_topic,age_segment,dt,visits
4548,1045145,Женщины,Спорт,31-35,2019-09-24 18:55:00,5
11856,1052453,Красота,Семья,26-30,2019-09-24 18:56:00,10
81,1040678,Деньги,Деньги,36-40,2019-09-24 18:59:00,6
17116,1057713,Отношения,Россия,26-30,2019-09-24 18:28:00,1
23538,1064135,Рассказы,Ремонт,31-35,2019-09-24 18:56:00,3
22104,1062701,Путешествия,Музыка,41-45,2019-09-24 18:55:00,1
13345,1053942,Наука,Еда,18-25,2019-09-24 19:00:00,4


В агрегированном датафрейме более 30 тысяч строк, дубликатов не обнаружено, располагаем данными за 24 сентября 2019 года, за период с 18:28 по 19:00.

## Дашборд доступен по ссылке ниже

![title](dash_prev.jpg)


Ссылка на дашборд:
https://public.tableau.com/profile/ilia3407#!/vizhome/Zen_dash/_?publish=yes
